In [1]:
from easing import easing
import pandas as pd
import numpy as np
from math import pi, radians, sin, cos, atan2, sqrt

In [2]:
def sph2cart(lon, lat, om=1.):
    """ Transforms spherical to cartesian coordinates
    
    Parameters
    ----------
    lat : array[1xn]
        Latitude in [degrees]
    lon : array[1xn]
        Longitude in [degrees]
    om : array[1xn], optional
        Angular magnitude in [degrees] or [degrees/Myr].
    
    Returns
    ----------
    x,y,z : floats
        Cartesian coordinates in same unit of measurement as 
        om ([degrees] or [degrees/Myr]).
    """
    
    m = np.size(lat)

    x = np.zeros(m)
    y = np.zeros(m)
    z = np.zeros(m)
    
    for ii in range(m):
        
        if np.size(lat) > 1:
            
            if isinstance(lat, type(pd.Series(dtype="float64"))):
                
                lonSph = lon.iloc[ii]
                latSph = lat.iloc[ii]
                
                if np.size(om) > 1: # om is given and is a pd.Series
                    omScalar = om.iloc[ii]
            
            else:
                
                lonSph = lon[ii]
                latSph = lat[ii]
                
                if np.size(om) > 1: # om is given and is a list
                    omScalar = om[ii]
            
        else:
                
            lonSph = lon
            latSph = lat
        
        
        omScalar = om
        lonRad = radians(lonSph)
        latRad = radians(latSph)
        
        if np.size(lat) > 1:
            x[ii] = omScalar * cos(lonRad) * cos(latRad)
            y[ii] = omScalar * cos(latRad) * sin(lonRad)
            z[ii] = omScalar * sin(latRad)     
        
        else:
            x = omScalar * cos(lonRad) * cos(latRad)
            y = omScalar * cos(latRad) * sin(lonRad)
            z = omScalar * sin(latRad)  
    
    return x, y, z


def cart2sph(x=[], y=[], z=[], 
             xyz = [],
             degreesFormat = True,
             onlyOm = False): 
    
    """ Transforms cartesian to spherical coordinates
    
    Parameters
    ----------
    x,y,z : array[1xn]
        Cartesian coordinates
    degreesFormat : boolean, optional
        Allows for output in degrees (True) or radians (False)
    
    Returns
    ----------
    lat : list if floats
        Latitude in degrees
    lon : list if floats
        Longitude in degrees
    om : list if floats
        Angle (magnitude) in degrees
        
    """
    
    # If variable xyz is not empty, and array with the three cartesian was given
    if len(xyz) != 0:
        if isinstance(xyz, (tuple, list, np.ndarray)):
            x = xyz[0]
            y = xyz[1]
            z = xyz[2]
            
        elif isinstance(xyz, pd.core.frame.DataFrame):
            x = xyz["x"]
            y = xyz["y"]
            z = xyz["z"]
        else:
            print ('xyz type not recognised')
      
            
    # If x, y and z are integers
    if isinstance(x, (int, float)):

        om = sqrt( (x**2 + y**2) + z**2 )
        lat = atan2( z, sqrt(x**2 + y**2) )
        lon = atan2(y, x)
        
        # Turns lat, lon output from radians to degrees        
        if degreesFormat:
            lat = np.round(np.degrees(lat), 7)
            lon = np.round(np.degrees(lon), 7)
            #if lon < 0:
                #lon = lon + 360
    
    
    else:
        # Set lat, lon and omega empty lists
        lat = []
        lon = []    
        om = []
               
        if onlyOm == False:
            
            # Iterate and extend lists
            lat.extend( [atan2(Az, sqrt(Ax**2 + Ay**2)) for Ax, Ay, Az in zip(x,y,z)] )
            lon.extend( [atan2(Ay, Ax) for Ax, Ay in zip(x,y)] )
            
            # Turns lat, lon output from radians to degrees        
            if degreesFormat:
                lat = list(np.degrees(lat))
                lon = list(np.degrees(lon))
                
        #for i in range(len(lon)):
            #if lon[i] < 0:
                #lon[i] = lon[i] + 360
        
        
        om.extend( [sqrt(Ax**2 + Ay**2 + Az**2) for Ax, Ay, Az in zip(x,y,z)] )
        
    return lon, lat, om


In [3]:
def spherical_interpolation(p0, p1, radius, t):
    """ Interpolate between two points on a sphere."""

    p0 = np.array(sph2cart(p0[0], p0[1], radius))
    p1 = np.array(sph2cart(p1[0], p1[1], radius))
    omega = np.arccos(np.dot(p0/np.linalg.norm(p0), p1/np.linalg.norm(p1)))
    so = np.sin(omega)
    x, y, z = np.sin((1.0-t)*omega) / so * p0 + np.sin(t*omega)/so * p1
    return cart2sph(x, y, z)[:2]

In [4]:
def geodesic_distance(point1Lon, point1Lat, point2Lon, point2Lat, radius):
    """ Calculates the geodesic distance between two point on a sphere,
    based on the Vincenty inverse problem formula. """
    
    # turn input coordinates from sph to radians
    lat1, lon1 = radians(point1Lat), radians(point1Lon)
    lat2, lon2 = radians(point2Lat), radians(point2Lon)

    a = cos(lat2)*sin(abs(lon2 - lon1))
    b = cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(abs(lon2 - lon1))
    c = sin(lat1)*sin(lat2) + cos(lat1)*cos(lat2)*cos(abs(lon2 - lon1))
    
    # geodetic distance in meters
    return radius * atan2( sqrt(a*a+b*b),c )

In [6]:
# Apply easing between 0 and 1, based on a power methods
# Power exponent n=4, frames per transition fpt=60
data_df = pd.DataFrame(np.array([[0.0, 0],[1.0, 0]]))
eased_data = easing.Eased(data_df, in_t=[0,1], wrap=False)
eased_data.power_ease(n=4, fpt=60)
eased_coords = eased_data.eased
eased_sample = eased_coords[:,0]


# Point coordinates:
p0 = -74.007, 40.712  # First point
p1 = -52.712, 47.562  # Second point
radius = 6371000

for t in eased_sample:
    
    p2 = spherical_interpolation(p0, p1, radius, t)
    
    point1Lon, point1Lat = p0
    point2Lon, point2Lat = p2
    radius = 6371000
    
    dist_02 = geodesic_distance(point1Lon, point1Lat, point2Lon, point2Lat, radius)
    print(p2[0], p2[1], dist_02/1000)

-74.007 40.712 0.0
-74.0069876 40.7120057 0.0012223070497080035
-74.0068015 40.7120917 0.019592987333349174
-74.0059953 40.712464 0.09916107078274894
-74.0038245 40.7134666 0.3134147324980281
-73.999247 40.7155804 0.7651768339625843
-73.9909225 40.7194236 1.5866590571228794
-73.9772115 40.7257515 2.9394973352115157
-73.9561748 40.7354548 5.014644802288984
-73.9255706 40.7495592 8.03248445788795
-73.882852 40.7692233 12.242776453472935
-73.8251623 40.7957354 17.92465042264409
-73.7493288 40.8305099 25.386619951080412
-73.6518541 40.8750819 34.96659595584585
-73.5289053 40.9311001 47.03186046075943
-73.3762996 41.0003175 61.97906336536376
-73.1894856 41.0845798 80.2342694429851
-72.9635216 41.1858108 102.2529039892144
-72.693047 41.3059935 128.51978657033874
-72.3722489 41.4471473 159.54910921327723
-71.9948216 41.6113002 195.88444927769666
-71.5539176 41.8004537 238.09878248934416
-71.0420915 42.0165409 286.79442828158557
-70.4512322 42.2613752 342.6031337345111
-69.7724867 42.5365877 4